In [1]:
import pandas as pd

# 데이터 불러오기
url = 'https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/datasets/vgames2.csv'
file = pd.read_csv(url)
data = file.iloc[:,1:]
data = data.dropna()

# Sales 데이터에서 'K'와 'M'를 곱셈으로 해결
data['NA_Sales'] = data['NA_Sales'].map(lambda x : x.replace('K','*1000'))
data['NA_Sales'] = data['NA_Sales'].map(lambda x : x.replace('M','*1000000'))

data['EU_Sales'] = data['EU_Sales'].map(lambda x : x.replace('K','*1000'))
data['EU_Sales'] = data['EU_Sales'].map(lambda x : x.replace('M','*1000000'))

data['JP_Sales'] = data['JP_Sales'].map(lambda x : x.replace('K','*1000'))
data['JP_Sales'] = data['JP_Sales'].map(lambda x : x.replace('M','*1000000'))

data['Other_Sales'] = data['Other_Sales'].map(lambda x : x.replace('K','*1000'))
data['Other_Sales'] = data['Other_Sales'].map(lambda x : x.replace('M','*1000000'))

def strTofloat(obj):
  try:
    num1 = float(obj.split('*')[0])
    num2 = float(obj.split('*')[1])
    prod = num1*num2
    return prod
  except:
    return float(obj)

data['NA_Sales'] = data['NA_Sales'].apply(strTofloat)
data['EU_Sales'] = data['EU_Sales'].apply(strTofloat)
data['JP_Sales'] = data['JP_Sales'].apply(strTofloat)
data['Other_Sales'] = data['Other_Sales'].apply(strTofloat)

# Name : 게임의 이름입니다.
# Platform : 게임이 지원되는 플랫폼의 이름입니다.
# Year : 게임이 출시된 연도입니다.
# Genre : 게임의 장르입니다.
# Publisher : 게임을 제작한 회사입니다.
# NA_Sales : 북미지역에서의 출고량입니다.
# EU_Sales : 유럽지역에서의 출고량입니다.
# JP_Sales : 일본지역에서의 출고량입니다.
# Other_Sales : 기타지역에서의 출고량입니다.

In [26]:
##### Q2. 연도별 게임의 트렌드가 있을까 #####

# 문제있는 연도자료 수정하기
def makeYear(num):
  num = int(num)
  if num<20:
    num+=2000
    return num
  elif 20<num<100:
    num+=1900
    return num
  else:
    return num

data['Year'] = data['Year'].apply(makeYear)
data['Total_Sales'] = data['NA_Sales']+data['EU_Sales']+data['JP_Sales']+data['Other_Sales']

gen_year = data.pivot_table(values='Total_Sales', index='Year', columns='Genre', aggfunc=sum)
gen_year = gen_year.fillna(0)
gen_year = pd.DataFrame(gen_year)

# 누적합계 계산해서 열 생성하는 함수
def cumsum(dataframe):
  dataframe['Action_cum'] = dataframe['Action'].cumsum()
  dataframe['Adventure_cum'] = dataframe['Adventure'].cumsum()
  dataframe['Fighting_cum'] = dataframe['Fighting'].cumsum()
  dataframe['Misc_cum'] = dataframe['Misc'].cumsum()
  dataframe['Platform_cum'] = dataframe['Platform'].cumsum()
  dataframe['Puzzle_cum'] = dataframe['Puzzle'].cumsum()
  dataframe['Racing_cum'] = dataframe['Racing'].cumsum()
  dataframe['Role-Playing_cum'] = dataframe['Role-Playing'].cumsum()
  dataframe['Shooter_cum'] = dataframe['Shooter'].cumsum()
  dataframe['Simulation_cum'] = dataframe['Simulation'].cumsum()
  dataframe['Sports_cum'] = dataframe['Sports'].cumsum()
  dataframe['Strategy_cum'] = dataframe['Strategy'].cumsum()

cumsum(gen_year)

In [3]:
pip install chart_studio

In [4]:
pip install plotly==4.8

In [32]:
import chart_studio.plotly as py
import chart_studio
import plotly.graph_objects as go

chart_studio.tools.set_credentials_file(username='Godot94', api_key='rkhDz3WA661P8ipQhPWx')
pd.options.plotting.backend = "plotly"

def cumplot(dataframe, title):
  data_cum = dataframe[['Action_cum', 'Adventure_cum', 'Fighting_cum', 'Misc_cum', 'Platform_cum', 'Puzzle_cum',
                         'Racing_cum', 'Role-Playing_cum', 'Shooter_cum', 'Simulation_cum', 'Sports_cum', 'Strategy_cum']]
  fig = data_cum.plot(title=title)
  fig.update_layout(legend_title_text='Genre')
  fig.show()

In [33]:
# 연도별 트렌드를 지역별로 알아보기
dt_NA = data[['Year','Genre','NA_Sales']]
dt_EU = data[['Year','Genre','EU_Sales']]
dt_JP = data[['Year','Genre','JP_Sales']]
dt_Other = data[['Year','Genre','Other_Sales']]

cumplot(gen_year_cum, 'Cumulative Sales - All Region')

NA_gen_year = data.pivot_table(values='NA_Sales', index='Year', columns='Genre', aggfunc=sum)
NA_gen_year = gen_year.fillna(0)
NA_gen_year = pd.DataFrame(gen_year)

cumsum(NA_gen_year)
cumplot(NA_gen_year, 'Cumulative Sales - North America')

EU_gen_year = data.pivot_table(values='EU_Sales', index='Year', columns='Genre', aggfunc=sum)
EU_gen_year = gen_year.fillna(0)
EU_gen_year = pd.DataFrame(gen_year)

cumsum(EU_gen_year)
cumplot(EU_gen_year, 'Cumulative Sales - Europe')

JP_gen_year = data.pivot_table(values='JP_Sales', index='Year', columns='Genre', aggfunc=sum)
JP_gen_year = gen_year.fillna(0)
JP_gen_year = pd.DataFrame(gen_year)

cumsum(JP_gen_year)
cumplot(JP_gen_year, 'Cumulative Sales - Japan')

Other_gen_year = data.pivot_table(values='Other_Sales', index='Year', columns='Genre', aggfunc=sum)
Other_gen_year = gen_year.fillna(0)
Other_gen_year = pd.DataFrame(gen_year)

cumsum(Other_gen_year)
cumplot(Other_gen_year, 'Cumulative Sales - Other Region')